# Representing biochemical systems with chemical hypergraphs

Here we show how to use chemical hypergraphs to represent biochemical systems. This representation is equivalent to that provided by chemical reaction networks, and by stochastic Petri nets.

In [1]:
using HyperGraphs

## Example 1: MEK-ERK dynamics

The model of MEK-ERK dynamics we use is adapted from

> Filippi, S. et al. Robustness of MEK-ERK Dynamics and Origins of Cell-to-Cell Variability in MAPK Signaling. _Cell Reports_ **15**, 2524–2535 (2016). [[doi]](http://dx.doi.org/10.1016/j.celrep.2016.05.024)

Note that parameter names have been changed.

We use [Symbolics.jl](https://github.com/JuliaSymbolics/Symbolics.jl) to represent model parameters and state variables as symbolics variables, which we use to generate symbolic equations.

In [2]:
using Symbolics

In [3]:
@variables t k₁ k₂ k₃ k₄ k₅ k₆ k′₁ k′₂ k′₃ k′₄ k′₅ k′₆;
@variables E(t) M(t) pE(t) ppE(t) Pt(t) E_M(t) pE_M(t) pE_Pt(t) ppE_Pt(t);

In [4]:
const ChE = ChemicalHyperEdge

rxs =  [# phosphorylation reactions
        ChE([E, M], [E_M], k₁),        # binding
        ChE([E_M], [E, M], k₂),        # unbinding
        ChE([E_M], [pE, M], k₃),       # phosphorylation #1
        ChE([pE, M], [pE_M], k₄),      # binding
        ChE([pE_M], [pE, M], k₅),      # unbinding
        ChE([pE_M], [ppE, M], k₆),     # phosphorylation #2

        # dephosphorylation reactions
        ChE([ppE, Pt], [ppE_Pt], k′₁), # binding
        ChE([ppE_Pt], [ppE, Pt], k′₂), # unbinding
        ChE([ppE_Pt], [pE, Pt], k′₃),  # dephosphorylation #1
        ChE([pE, Pt], [pE_Pt], k′₄),   # binding
        ChE([pE_Pt], [pE, Pt], k′₅),   # unbinding
        ChE([pE_Pt], [E, Pt], k′₆)]    # dephosphorylation #2

12-element Vector{ChemicalHyperEdge{Num}}:
 ChemicalHyperEdge{Num}(SpeciesSet{Num}(Num[E(t), M(t)], [1, 1]), SpeciesSet{Num}(Num[E_M(t)], [1]), k₁)
 ChemicalHyperEdge{Num}(SpeciesSet{Num}(Num[E_M(t)], [1]), SpeciesSet{Num}(Num[E(t), M(t)], [1, 1]), k₂)
 ChemicalHyperEdge{Num}(SpeciesSet{Num}(Num[E_M(t)], [1]), SpeciesSet{Num}(Num[pE(t), M(t)], [1, 1]), k₃)
 ChemicalHyperEdge{Num}(SpeciesSet{Num}(Num[pE(t), M(t)], [1, 1]), SpeciesSet{Num}(Num[pE_M(t)], [1]), k₄)
 ChemicalHyperEdge{Num}(SpeciesSet{Num}(Num[pE_M(t)], [1]), SpeciesSet{Num}(Num[pE(t), M(t)], [1, 1]), k₅)
 ChemicalHyperEdge{Num}(SpeciesSet{Num}(Num[pE_M(t)], [1]), SpeciesSet{Num}(Num[ppE(t), M(t)], [1, 1]), k₆)
 ChemicalHyperEdge{Num}(SpeciesSet{Num}(Num[ppE(t), Pt(t)], [1, 1]), SpeciesSet{Num}(Num[ppE_Pt(t)], [1]), k′₁)
 ChemicalHyperEdge{Num}(SpeciesSet{Num}(Num[ppE_Pt(t)], [1]), SpeciesSet{Num}(Num[ppE(t), Pt(t)], [1, 1]), k′₂)
 ChemicalHyperEdge{Num}(SpeciesSet{Num}(Num[ppE_Pt(t)], [1]), SpeciesSet{Num}(Num[pE(t), Pt(t)]

In [5]:
X = ChemicalHyperGraph(rxs)

ChemicalHyperGraph{Num}(Num[E(t), M(t), E_M(t), pE(t), pE_M(t), ppE(t), Pt(t), ppE_Pt(t), pE_Pt(t)], ChemicalHyperEdge{Num}[ChemicalHyperEdge{Num}(SpeciesSet{Num}(Num[E(t), M(t)], [1, 1]), SpeciesSet{Num}(Num[E_M(t)], [1]), k₁), ChemicalHyperEdge{Num}(SpeciesSet{Num}(Num[E_M(t)], [1]), SpeciesSet{Num}(Num[E(t), M(t)], [1, 1]), k₂), ChemicalHyperEdge{Num}(SpeciesSet{Num}(Num[E_M(t)], [1]), SpeciesSet{Num}(Num[pE(t), M(t)], [1, 1]), k₃), ChemicalHyperEdge{Num}(SpeciesSet{Num}(Num[pE(t), M(t)], [1, 1]), SpeciesSet{Num}(Num[pE_M(t)], [1]), k₄), ChemicalHyperEdge{Num}(SpeciesSet{Num}(Num[pE_M(t)], [1]), SpeciesSet{Num}(Num[pE(t), M(t)], [1, 1]), k₅), ChemicalHyperEdge{Num}(SpeciesSet{Num}(Num[pE_M(t)], [1]), SpeciesSet{Num}(Num[ppE(t), M(t)], [1, 1]), k₆), ChemicalHyperEdge{Num}(SpeciesSet{Num}(Num[ppE(t), Pt(t)], [1, 1]), SpeciesSet{Num}(Num[ppE_Pt(t)], [1]), k′₁), ChemicalHyperEdge{Num}(SpeciesSet{Num}(Num[ppE_Pt(t)], [1]), SpeciesSet{Num}(Num[ppE(t), Pt(t)], [1, 1]), k′₂), ChemicalHyperE

Here we show a simple implementation of mass action kinetics on both a single chemical hyperedge and on a chemical hypergraph. These abstract functions may then be used to generate the system of equations describing the deterministic dynamics of the system.

In [6]:
mass_action(e::ChemicalHyperEdge) = rate(e) * prod(src(e) .^ src_stoich(e))
mass_action(x::ChemicalHyperGraph) = mass_action.(hyperedges(x))

mass_action (generic function with 2 methods)

In [7]:
lhss = [Differential(t)(v) for v in vertices(X)]
rhss = incidence_matrix(X) * mass_action(X)
eqs  = Equation.(lhss, rhss)

9-element Vector{Equation}:
 Differential(t)(E(t)) ~ k′₆*pE_Pt(t) + k₂*E_M(t) - k₁*E(t)*M(t)
 Differential(t)(M(t)) ~ k₂*E_M(t) + k₃*E_M(t) + k₅*pE_M(t) + k₆*pE_M(t) - k₁*E(t)*M(t) - k₄*M(t)*pE(t)
 Differential(t)(E_M(t)) ~ k₁*E(t)*M(t) - k₂*E_M(t) - k₃*E_M(t)
 Differential(t)(pE(t)) ~ k′₃*ppE_Pt(t) + k₃*E_M(t) + k₅*pE_M(t) + k′₅*pE_Pt(t) - k₄*M(t)*pE(t) - k′₄*Pt(t)*pE(t)
 Differential(t)(pE_M(t)) ~ k₄*M(t)*pE(t) - k₅*pE_M(t) - k₆*pE_M(t)
 Differential(t)(ppE(t)) ~ k′₂*ppE_Pt(t) + k₆*pE_M(t) - k′₁*Pt(t)*ppE(t)
 Differential(t)(Pt(t)) ~ k′₂*ppE_Pt(t) + k′₃*ppE_Pt(t) + k′₅*pE_Pt(t) + k′₆*pE_Pt(t) - k′₁*Pt(t)*ppE(t) - k′₄*Pt(t)*pE(t)
 Differential(t)(ppE_Pt(t)) ~ k′₁*Pt(t)*ppE(t) - k′₂*ppE_Pt(t) - k′₃*ppE_Pt(t)
 Differential(t)(pE_Pt(t)) ~ k′₄*Pt(t)*pE(t) - k′₅*pE_Pt(t) - k′₆*pE_Pt(t)

These symbolic equations may then be simulated using other packages in the Julia package ecosystem (e.g. [ModelingToolkit.jl](https://github.com/SciML/ModelingToolkit.jl) and [OrdinaryDiffEq.jl](https://github.com/SciML/OrdinaryDiffEq.jl)).